<a href="https://colab.research.google.com/github/leeseungchae/watching_you_chatbot/blob/SeungChae/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-m41grs1j
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-m41grs1j


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from sklearn.model_selection import train_test_split 

In [ ]:
device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
csv = pd.read_csv('/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/02_preprocessing/bert_preprocessing.csv') 
trainData, testData = train_test_split(csv, test_size = 0.3, random_state=42) 

In [ ]:
df1 = pd.DataFrame(trainData)
df1.to_csv('/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/02_preprocessing/bert_preprocessing_train.tsv',index=False,
               sep='\t',encoding='utf-8')
df2 = pd.DataFrame(testData)
df2.to_csv('/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/02_preprocessing/bert_preprocessing_test.tsv',index=False,
               sep='\t',encoding='utf-8')



In [ ]:
dataset_train = nlp.data.TSVDataset("/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/02_preprocessing/bert_preprocessing_train.tsv", field_indices=[0,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/02_preprocessing/bert_preprocessing_test.tsv", field_indices=[0,2], num_discard_samples=1)

In [ ]:
dataset_train[0]

['그리고 차가운 것으로 주세요', '0']

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 15
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=126,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [ ]:
device = torch.device("cuda:0")
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/34 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 4.800900936126709 train acc 0.015625
epoch 1 train acc 0.13117121848739496


  0%|          | 0/15 [00:00<?, ?it/s]

epoch 1 test acc 0.3737938596491228


  0%|          | 0/34 [00:00<?, ?it/s]

...

epoch 14 test acc 0.7512061403508772


  0%|          | 0/34 [00:00<?, ?it/s]

epoch 15 batch id 1 loss 0.9984994530677795 train acc 0.765625
epoch 15 train acc 0.8385854341736695


  0%|          | 0/15 [00:00<?, ?it/s]

epoch 15 test acc 0.7522478070175438


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/03_DeepLearning/bert.pt')


In [ ]:
!wget -O .cache/ratings_train.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
# dataset_train = nlp.data.TSVDataset("/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/02_preprocessing/bert_preprocessing.tsv", field_indices=[0,2], num_discard_samples=1)

--2022-05-19 09:12:07--  http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
Resolving skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)... 52.219.60.34
Connecting to skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)|52.219.60.34|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘.cache/ratings_train.txt’

.cache/ratings_trai 100%[===================>]  13.95M  14.3MB/s    in 1.0s    

2022-05-19 09:12:08 (14.3 MB/s) - ‘.cache/ratings_train.txt’ saved [14628807/14628807]



In [ ]:
# for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):

In [ ]:
new_test = nlp.data.TSVDataset(".cache/ratings_train.txt" , field_indices=[1,2], num_discard_samples=1)
test_set = BERTDataset(new_test , 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=4)

In [ ]:
# for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)): 
#   token_ids = token_ids.long().to(device) 
#   segment_ids = segment_ids.long().to(device) 
#   valid_length= valid_length 
#   out = model(token_ids, valid_length, segment_ids)
#   prediction = out.cpu().detach().numpy().argmax()

#   batch_id = int(batch_id)
#   # prediction = float(prediction)

#   print(f"{batch_id }번째 문장의 분류 예측값은 {prediction} 입니다.")
#   # print(f"{batch_id} "+"번째 문장의 분류 예측값은"+" {prediction}"+"입니다.")

#   # print(f"{batch_id} "+"번째 문장의 분류 예측값은"+ {prediction} "+" 입니다.")

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/02_preprocessing/bert_preprocessing.csv')
df

,SENTENCE,MAIN,num
0,아이스아메리카노 하나요,일반주문,0
1,네 텀블러에 넣어주세요,텀블러사용,1
2,그란데 사이즈로 주세요,일반주문,0
3,저 카푸치노로 주문할게요,일반주문,0
4,그럼 2000원 추가해서 브레드도 같이 시킬게요,결제요청,2
...,...,...,...
3043,하우스 블렌딩은 뭐예요?,메뉴문의,8
3044,약간 산미 나는 걸로 한 잔 주실래요?,일반주문,0
3045,마시고 가면 얼마예요?,일반주문,0
3046,영수증 하나 주실래요?,영수증요청,45


In [ ]:
category_list = list(df['MAIN'].unique())
category_list
cat_len = len(category_list)

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    if len(predict_sentence) <5 :
        print("너무 짧아요")
        return False
    else:




      data = [predict_sentence, '0']
      dataset_another = [data]

      another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
      test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
      
      model.eval()

      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)

          valid_length= valid_length
          label = label.long().to(device)

          out = model(token_ids, valid_length, segment_ids)
          prediction = out.cpu().detach().numpy().argmax()

          test_eval=[]
          test_eval_2=[]
          for i in out:
              logits=i
              logits = logits.detach().cpu().numpy()
          for k , c in enumerate(category_list):
          # for k,(c,p) in enumerate(zip(category_list,prediction)):
              
              if np.argmax(logits) == k:
                  value = logits[k]
                  value  = round(value*10)
                  if value <50 :
                      print("죄송합니다 다시 말씀해주세요")
                      print(value)
                      return False
                  else :
                      test_eval.append(c)
                      value = logits[k]
                      test_eval_2.append(value)

              # elif np.argmax(logits) == 1:
              #     test_eval.append("텀블라사용")

          print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")
          # print(value)
          print(test_eval_2[0])



using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요(5자 이상) : ")
    if sentence == 0 :
        
        break
    predict(sentence)


    print("\n")

하고싶은 말을 입력해주세요(5자 이상) : 안녕하세요


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


죄송합니다 다시 말씀해주세요
38


하고싶은 말을 입력해주세요(5자 이상) : 아아 하나주세요
>> 입력하신 내용에서 일반주문 느껴집니다.
8.15798


하고싶은 말을 입력해주세요(5자 이상) : 라떼 주세요
>> 입력하신 내용에서 일반주문 느껴집니다.
8.513828




KeyboardInterrupt: ignored